In [13]:
from itertools import chain, combinations
import numpy as np # Fundamental package for scientific computing with Python
import pandas as pd
import calendar
import time
from sklearn.metrics import accuracy_score, precision_score, f1_score
from lib.Utility import computeEquity, getData
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


commissioni = 0.0008


In [14]:
# Train the model
def trainModel(X_train, y_train):
    # Set regularization rate
    reg = 0.01
    # train a logistic regression model on the training set
    mod = LogisticRegression(C=1/reg, solver="liblinear", random_state=42).fit(X_train, y_train)

    return mod

In [15]:
data = getData()

features = ['feat_Open', 'feat_vixOpen', 'feat_vixPrevClose', 'feat_vvixOpen', 'feat_vvixPrevClose', 'feat_Acc', 'feat_PrevCluster', 'feat_bigBodyRange']
combinazioni = chain.from_iterable(combinations(features, r) for r in range(1, len(features) + 1))

combos=[]
for idx,combinazione in enumerate(combinazioni):
  current_GMT = time.gmtime()
  time_stamp = calendar.timegm(current_GMT)

  print(idx,time.strftime("%H:%M:%S", time.localtime()), ':', combinazione)
  combos.append(combinazione)

dataframe = pd.DataFrame(columns=['DataSetName', 'TrainAccuracy', 'TrainPrecision', 'TrainF2','TrainRecall','TrainROC' ,'TestAccuracy', 'TestPrecision', 'TestF2','TestRecall','TestROC', 'BestParams'])

split_test_date = '2019-02-21'
start_date = '1990-09-04'

# Split Dataset
print("Performing splitting operations....")
# train_data = data[(data['Date'] < split_test_date) & (data['Date'] > start_date)].reset_index(drop=True)
# val_data = data[(data['Date'] < split_test_date) & (data['Date'] > start_date)].reset_index(drop=True)
# test_data = data[data['Date'] >= split_test_date].reset_index(drop=True)

# Select 'feat' columns
print("Extracting features columns and creating target variable...")

print(combos)

for idx,combo in enumerate(combos):
  current_GMT = time.gmtime()
  time_stamp = calendar.timegm(current_GMT)
  print(idx, time.strftime("%H:%M:%S", time.localtime()),'Trying with following combo', combo)
  feat_cols = [col for col in combo if 'feat' in col]
  # Import Training and Validation Data
  # x_train = train_data[feat_cols]
  # y_train = np.where(train_data['Close'] > train_data['Open'], 1, -1)

  # x_test = test_data[feat_cols]
  # y_test = np.where(test_data['Close'] > test_data['Open'], 1, -1)

  x_train, x_test, y_train, y_test = train_test_split(data[feat_cols], data['Cluster'], test_size= 0.3, random_state= 0)

  x_train = x_train.sort_index()
  x_test = x_test.sort_index()
  y_train= y_train.sort_index()
  y_test = y_test.sort_index()

  model = trainModel(x_train, y_train)

  # Predictions
  y_train_pred = model.predict(x_train.sort_index())
  y_test_pred = model.predict(x_test.sort_index())

  # print('y_test_pred', y_test_pred)
  # clf.fit(x_train,y_train)

  # y_train_pred = clf.predict(x_train)
  # y_test_pred = clf.predict(x_test)


  #VN Calcolo la equity

  test_data = data[data.index.isin(x_test.index)]
  # print('test_data',test_data)

  test_data['cluster'] = y_test_pred
  test_equity, test_cagr= computeEquity(test_data, commissioni, feat_cols)

  print("Train metrics...")
  # Metrics
  train_accuracy = accuracy_score(y_train, y_train_pred)
  train_precision = precision_score(y_train, y_train_pred)

  print("Test metrics...")
  test_accuracy = accuracy_score(y_test, y_test_pred)
  test_precision = precision_score(y_test, y_test_pred)
  test_f1 = f1_score(y_test, y_test_pred)

  # Save Results
  new_row = pd.Series({
      'DataSetName': combo,
      'TrainAccuracy': train_accuracy,
      'TrainPrecision': train_precision,
      'TestAccuracy': test_accuracy,
      'TestPrecision': test_precision,
      'TestF1': test_f1,
      'BestParams': model.get_params,
      'TestEquity': test_equity,
      'TestCagr': test_cagr
  })
  print(new_row)
  dataframe=pd.concat([dataframe, new_row.to_frame().T], ignore_index=True)

dataframe.to_excel("Dataframe/"+str(time_stamp) + ".xlsx")

# print(result)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
C:\Users\vitto\AppData\Local\Temp\ipykernel_10512\820412064.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['cluster'] = y_test_pred


0 21:16:15 : ('feat_Open',)
1 21:16:15 : ('feat_vixOpen',)
2 21:16:15 : ('feat_vixPrevClose',)
3 21:16:15 : ('feat_vvixOpen',)
4 21:16:15 : ('feat_vvixPrevClose',)
5 21:16:15 : ('feat_Acc',)
6 21:16:15 : ('feat_PrevCluster',)
7 21:16:15 : ('feat_bigBodyRange',)
8 21:16:15 : ('feat_Open', 'feat_vixOpen')
9 21:16:15 : ('feat_Open', 'feat_vixPrevClose')
10 21:16:15 : ('feat_Open', 'feat_vvixOpen')
11 21:16:15 : ('feat_Open', 'feat_vvixPrevClose')
12 21:16:15 : ('feat_Open', 'feat_Acc')
13 21:16:15 : ('feat_Open', 'feat_PrevCluster')
14 21:16:15 : ('feat_Open', 'feat_bigBodyRange')
15 21:16:15 : ('feat_vixOpen', 'feat_vixPrevClose')
16 21:16:15 : ('feat_vixOpen', 'feat_vvixOpen')
17 21:16:15 : ('feat_vixOpen', 'feat_vvixPrevClose')
18 21:16:15 : ('feat_vixOpen', 'feat_Acc')
19 21:16:15 : ('feat_vixOpen', 'feat_PrevCluster')
20 21:16:15 : ('feat_vixOpen', 'feat_bigBodyRange')
21 21:16:15 : ('feat_vixPrevClose', 'feat_vvixOpen')
22 21:16:15 : ('feat_vixPrevClose', 'feat_vvixPrevClose')
23 21

C:\Users\vitto\AppData\Local\Temp\ipykernel_10512\820412064.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['cluster'] = y_test_pred


Train metrics...
Test metrics...
DataSetName                                         (feat_vixOpen,)
TrainAccuracy                                              0.540532
TrainPrecision                                             0.540333
TestAccuracy                                               0.545176
TestPrecision                                              0.545098
TestF1                                                     0.700605
BestParams        <bound method BaseEstimator.get_params of Logi...
TestEquity                                              -815.734649
TestCagr                                                -149.905295
dtype: object
2 21:16:16 Trying with following combo ('feat_vixPrevClose',)
Equity: -729.7549126464855


C:\Users\vitto\AppData\Local\Temp\ipykernel_10512\820412064.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['cluster'] = y_test_pred


...

Train metrics...
Test metrics...
DataSetName       (feat_Open, feat_vixPrevClose, feat_vvixOpen, ...
TrainAccuracy                                              0.637676
TrainPrecision                                             0.632122
TestAccuracy                                               0.650842
TestPrecision                                              0.645235
TestF1                                                     0.711392
BestParams        <bound method BaseEstimator.get_params of Logi...
TestEquity                                              5218.100678
TestCagr                                                 958.915898
dtype: object
253 21:18:46 Trying with following combo ('feat_vixOpen', 'feat_vixPrevClose', 'feat_vvixOpen', 'feat_vvixPrevClose', 'feat_Acc', 'feat_PrevCluster', 'feat_bigBodyRange')
Equity: 5080.10214301758


C:\Users\vitto\AppData\Local\Temp\ipykernel_10512\820412064.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['cluster'] = y_test_pred


Train metrics...
Test metrics...
DataSetName       (feat_vixOpen, feat_vixPrevClose, feat_vvixOpe...
TrainAccuracy                                              0.642271
TrainPrecision                                             0.637587
TestAccuracy                                               0.647779
TestPrecision                                              0.645955
TestF1                                                     0.705506
BestParams        <bound method BaseEstimator.get_params of Logi...
TestEquity                                              5080.102143
TestCagr                                                  933.55629
dtype: object
254 21:18:46 Trying with following combo ('feat_Open', 'feat_vixOpen', 'feat_vixPrevClose', 'feat_vvixOpen', 'feat_vvixPrevClose', 'feat_Acc', 'feat_PrevCluster', 'feat_bigBodyRange')
Equity: 4914.682587353519


C:\Users\vitto\AppData\Local\Temp\ipykernel_10512\820412064.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['cluster'] = y_test_pred


Train metrics...
Test metrics...
DataSetName       (feat_Open, feat_vixOpen, feat_vixPrevClose, f...
TrainAccuracy                                              0.640958
TrainPrecision                                             0.636863
TestAccuracy                                               0.648545
TestPrecision                                              0.647754
TestF1                                                     0.704823
BestParams        <bound method BaseEstimator.get_params of Logi...
TestEquity                                              4914.682587
TestCagr                                                 903.157596
dtype: object
